# Scratch code for testing purposes and small experiments. 


In [31]:

import numpy as np
import importlib
from Graph import Graph
from helpers import *
import pandas as pd
import random
from tqdm import tqdm
from deterministic_attack import DeterministicAttack
from revisited_spectral import RevisitedSpectral
from erdos import SpectralAttack


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Exploring the spectral approach

In [32]:
dataset = "cora"
G = Graph.from_txt(f"datasets/{dataset}.txt")
G1, G2 = G.split_dataset(common_prop=0.0, graph1_prop=0.0)
A = np.dot(G.adj_matrix, G.adj_matrix.T)

print(G.adj_matrix.shape)
print(len(G.edges()))

(516, 516)
1188


## Deterministic -> Heuristic -> Deterministic 

In [33]:
deter = DeterministicAttack(G1, A)
deter.run()
Gstar = deter.get_reconstructed_graph()

proba = RevisitedSpectral(Gstar, A)
n_nodes = Gstar.adj_matrix.shape[0]
proba.run_bis(alpha=0.0, beta=0.0, gamma=0.0)
Gstar = proba.get_reconstructed_graph()
print("Errors after revisited:", np.sum(np.abs(Gstar.adj_matrix - G.adj_matrix)))
stats = ROC_stats(Gstar, G)
print("Before sanity check:", stats)
forgotten_slots = proba.sanity_check_with_high_loss()
Gstar = proba.get_reconstructed_graph()
stats = ROC_stats(Gstar, G)
print("After sanity check:", stats)

deter = DeterministicAttack(Gstar, A)
deter.run()
Gstar = deter.get_reconstructed_graph()
# proba = RevisitedSpectral(Gstar, A)
# proba.run(alpha=0.0, beta=0.0, gamma=0.0)
# Gstar = proba.get_reconstructed_graph()
# print("Errors after revisited:", np.sum(np.abs(Gstar.adj_matrix - G.adj_matrix)))
# stats = ROC_stats(Gstar, G)
# print("Before sanity check:", stats)
# forgotten_slots = proba.sanity_check_with_high_loss()
# Gstar = proba.get_reconstructed_graph()
# stats = ROC_stats(Gstar, G)
# print("After sanity check:", stats)
# deter = DeterministicAttack(Gstar, A)
# deter.run()
Gstar = deter.get_reconstructed_graph()
Gstar_fixed = Gstar.copy()
Gstar_fixed.fix_edges()
print("Errors :", np.sum(np.abs(Gstar_fixed.adj_matrix - G.adj_matrix)))




Triangle attack: 100%|██████████| 119/119 [00:00<00:00, 6793.37it/s]


+------+--------+
| Stat | Value  |
+------+--------+
|  0s  | 11186  |
|  1s  |  238   |
|  ?s  | 254832 |
+------+--------+ 



Triangle attack: 100%|██████████| 119/119 [00:00<00:00, 7065.22it/s]


+------+--------+
| Stat | Value  |
+------+--------+
|  0s  | 91928  |
|  1s  |  238   |
|  ?s  | 174090 |
+------+--------+ 



Triangle attack: 100%|██████████| 119/119 [00:00<00:00, 7734.73it/s]


...

+------+--------+
| Stat | Value  |
+------+--------+
|  0s  | 263822 |
|  1s  |  2362  |
|  ?s  |   72   |
+------+--------+ 



Triangle attack: 100%|██████████| 1181/1181 [00:00<00:00, 10131.69it/s]


+------+--------+
| Stat | Value  |
+------+--------+
|  0s  | 263840 |
|  1s  |  2362  |
|  ?s  |   54   |
+------+--------+ 



Triangle attack: 100%|██████████| 1181/1181 [00:00<00:00, 9775.12it/s]


+------+--------+
| Stat | Value  |
+------+--------+
|  0s  | 263840 |
|  1s  |  2362  |
|  ?s  |   54   |
+------+--------+ 

Errors : 14


In [34]:
stats = ROC_stats(Gstar_fixed, G)
print(stats)

(2362, 0, 263880, 14)


## Erdos method

In [35]:
proba = SpectralAttack(A)
proba.run()
Gstar = proba.get_reconstructed_graph()
print("Errors after erdos:", np.sum(np.abs(Gstar.adj_matrix - G.adj_matrix)))

100%|██████████| 516/516 [00:04<00:00, 110.68it/s]

Errors after erdos: 32


## Analysis 

In [36]:
# Identifying the components of the graph that are not reconstructed

A_prime = np.dot(Gstar_fixed.adj_matrix, Gstar_fixed.adj_matrix.T)
slots_of_error = np.argwhere(A != A_prime)
nodes_of_error = set()
for i, j in slots_of_error:
    nodes_of_error.add(i)
    nodes_of_error.add(j)

nodes_of_error = list(nodes_of_error)

components = {}
hubs = []
for node in nodes_of_error:
    if A[node, node] == A_prime[node, node]:
        hubs.append(node)


for hub in hubs:
    components[hub] = []
    for node in nodes_of_error:
        if A[hub, node] != A_prime[hub, node]:
            components[hub].append(node)


for hub, nodes in components.items():
    print(f" --- Hub ----: {hub}")
    for node in nodes:
        print(f"Node : {node}, degree in A : {A[node, node]}, degree in A_prime : {A_prime[node, node]}")

# G_prime_1 = Gstar_fixed.copy()
# G_prime_2 = Gstar_fixed.copy()

# for hub, nodes in components.items():
#     G_prime_1.add_edge((nodes[0], nodes[1]))
#     G_prime_1.add_edge((nodes[2], nodes[3]))    

#     G_prime_2.add_edge((nodes[0], nodes[3]))
#     G_prime_2.add_edge((nodes[1], nodes[2]))

#     if len(nodes) > 4:
#         G_prime_1.add_edge((nodes[4], nodes[5]))
#         G_prime_2.add_edge((nodes[4], nodes[5]))

# A_prime_1 = np.dot(G_prime_1.adj_matrix, G_prime_1.adj_matrix.T)
# A_prime_2 = np.dot(G_prime_2.adj_matrix, G_prime_2.adj_matrix.T)


# G_prime_1.to_txt(f"datasets/{dataset}_prime_1.txt")
# G_prime_2.to_txt(f"datasets/{dataset}_prime_2.txt")
# print("Errors after fixing:", np.sum(np.abs(A - A_prime_1)), np.sum(np.abs(A - A_prime_2)))


 --- Hub ----: 110
Node : 484, degree in A : 2, degree in A_prime : 1
Node : 337, degree in A : 2, degree in A_prime : 1
Node : 285, degree in A : 2, degree in A_prime : 1
Node : 445, degree in A : 2, degree in A_prime : 1
 --- Hub ----: 112
Node : 294, degree in A : 2, degree in A_prime : 1
Node : 233, degree in A : 2, degree in A_prime : 1
Node : 138, degree in A : 2, degree in A_prime : 1
Node : 367, degree in A : 2, degree in A_prime : 1
Node : 115, degree in A : 2, degree in A_prime : 1
Node : 116, degree in A : 2, degree in A_prime : 1
 --- Hub ----: 251
Node : 267, degree in A : 2, degree in A_prime : 1
Node : 333, degree in A : 2, degree in A_prime : 1
Node : 248, degree in A : 2, degree in A_prime : 1
Node : 126, degree in A : 2, degree in A_prime : 1


In [37]:

dataset = "bio-diseasome"
G_prime_1 = Graph.from_txt(f"datasets/{dataset}_prime_1.txt")
G_prime_2 = Graph.from_txt(f"datasets/{dataset}_prime_2.txt")

In [38]:
for hub, nodes in components.items():
    print(f" --- Hub ----: {hub}")
    for node in nodes:
        print(node, G.neighbors(node))

 --- Hub ----: 110
484 [110 445]
337 [110 285]
285 [110 337]
445 [110 484]
 --- Hub ----: 112
294 [112 233]
233 [112 294]
138 [112 367]
367 [112 138]
115 [112 116]
116 [112 115]
 --- Hub ----: 251
267 [126 251]
333 [248 251]
248 [251 333]
126 [251 267]
